In [10]:
import pandas as pd
import re
import plotly.graph_objects as go

# Funkcja do usunięcia litery na końcu wartości
def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)

# Wczytanie danych z pliku CSV
data = pd.read_csv('men_100m.csv')

# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.replace(',', '.').astype(float)

# Dodanie nowej kolumny 'Year' na podstawie kolumny 'Event Date'
data['Year'] = data['Event Date'].dt.year

# Filtrowanie danych od roku 1975
data = data[data['Year'] >= 1975]

# Grupowanie danych według roku i wybór najlepszego wyniku dla każdego roku
best_results = data.groupby('Year')['Time'].min()

# Tworzenie wykresu za pomocą Plotly
fig = go.Figure()

# Tworzenie słownika przypisującego kolor dla każdego zawodnika
colors = {
    'Usain Bolt': 'green',
    'Carl Lewis': 'red',
    'Robert Hayes': 'blue',
    'Jim Hines': 'orange',
    'Valeriy Borzov': 'purple',
    'Steve Riddick': 'cyan',
    'Hasely Crawford': 'magenta',
    'Silvio Leonard': 'lime',  
    'Clancy Edwards': 'yellow',
    'Pietro Mennea': 'pink',
    'James Sanford': 'brown',
    'Calvin Smith': 'gray',  
    'Mel Lattany': 'indigo',
    'Chidi Imoh': 'teal',
    'Leroy Burrell': 'olive',
    'Michael Marsh': 'navy',
    'Linford Christie': 'maroon',
    'Donovan Bailey': 'silver',
    'Maurice Greene': 'aqua',
    'Ato Boldon': 'fuchsia',
    'Justin Gatlin': 'gold',
    'Asafa Powell': 'violet',
    'Tyson Gay': 'salmon',
    'Christian Coleman': 'lightblue',
    'Michael Norman': 'black',
    'Trayvon Bromell': 'khaki',
    'Fred Kerley': 'lightgray',
    'Ferdinand Omanyala': 'tomato',
    'Patrick Johnson': 'brown',  
}


# Dodawanie punktów dla najlepszych wyników w każdym roku
for year, time in best_results.items():
    athlete_data = data.loc[(data['Year'] == year) & (data['Time'] == time)]
    athlete = athlete_data['Athlete'].values[0]
    country = athlete_data['Country'].values[0]
    color = colors.get(athlete, 'gray')
    label = f"Zawodnik: {athlete}<br>Rok: {int(year)}<br>Czas: {time} s<br>Kraj: {country}"
    fig.add_trace(go.Scatter(
        x=[year],
        y=[time],
        mode='markers',
        name=athlete,
        text=[label],
        hovertemplate='%{text}<extra></extra>',
        marker=dict(color=color, size=15),  # Zwiększenie rozmiaru kulek
    ))

# Dodawanie prostokąta 1
fig.add_shape(
    type="rect",
    x0=1981,
    y0=9.5,
    x1=1991,
    y1=10.5,
    fillcolor="rgba(255, 0, 0, 0.2)",
    line=dict(color='rgba(255, 0, 0, 0.2)'),
    opacity=0.5,
    layer="below",
    xref='x',
    yref='y'
)

# Dodawanie prostokąta 2
fig.add_shape(
    type="rect",
    x0=2008,
    y0=9.5,
    x1=2013,
    y1=10.5,
    fillcolor="rgba(0, 255, 0, 0.2)",
    line=dict(color='rgba(0, 255, 0, 0.2)'),
    opacity=0.5,
    layer="below",
    xref='x',
    yref='y'
)

# Konfiguracja układu wykresu
fig.update_layout(
    xaxis=dict(title='Rok'),
    yaxis=dict(title='Najlepszy czas (s)'),
    title='Najlepszy wynik w biegu na 100 m',
    showlegend=True,
    height=700
)

# Ustawienie skalowania osi Y
fig.update_yaxes(range=[9.5, 10.2])

# Zapisywanie wykresu do pliku HTML
fig.write_html('best_of_every_season.html')





In [17]:
import plotly.graph_objects as go
import pandas as pd
import networkx as nx
import random

data = {
    'Country': ['USA', 'JAM', 'GBR', 'TTO', 'CAN', 'NGR', 'RSA', 'FRA', 'SKN', 'JPN'],
    'Count': [1555, 696, 236, 187, 148, 146, 121, 109, 80, 78],
    'Picture': ['US.png', 'JM.png', 'GB.png', 'TT.png', 'CA.png', 'NG.png', 'ZA.png', 'FR.png', 'KN.png', 'JP.png']
}

# Wczytanie danych do DataFrame
df = pd.DataFrame(data)

# Tworzenie figury
fig = go.Figure()

# Tworzenie grafu
G = nx.Graph()

# Dodawanie wierzchołków do grafu
for i, row in df.iterrows():
    G.add_node(i)

# Generowanie losowego rozkładu kulek
pos = nx.spring_layout(G, iterations=2000, scale=0.5, seed=42)

# Dodawanie markerów z obrazkami i etykietami
for i, row in df.iterrows():
    x, y = pos[i]

    fig.add_trace(go.Scatter(
        x=[x],
        y=[y],
        mode='markers',
        marker=dict(
            size=30 * row['Count'] * 0.01,
            opacity=0.7,
            symbol='circle',
            color='rgba(0, 0, 0, 0)',
        ),
        hovertext=[f'Country: {row["Country"]}, Count: {row["Count"]}'],
        hoverinfo='text',
    ))
    fig.add_layout_image(
        dict(
            source=f"https://raw.githubusercontent.com/matahombres/CSS-Country-Flags-Rounded/master/flags/{row['Picture']}",
            xref="x",
            yref="y",
            x=x,
            y=y,
            sizex=0.001 * row['Count'],  # Mnożenie wartości Count przez 0.1
            sizey=0.001 * row['Count'],  # Mnożenie wartości Count przez 0.1
            xanchor="center",
            yanchor="middle",
            opacity=1,
            layer="above"
        )
    )

# Aktualizowanie osi x
fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False)

# Aktualizowanie osi y
fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False)

# Aktualizowanie układu wykresu
fig.update_layout(
    width=1300,
    height=750,
    plot_bgcolor="white",
    showlegend=False
)

# Zapisywanie wykresu do pliku HTML
fig.write_html("country_bubbles.html")



In [23]:
import pandas as pd
import plotly.express as px

# Funkcja do usunięcia litery na końcu wartości
def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)

# Wczytanie danych z pliku CSV
data = pd.read_csv('men_100m.csv')

# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.replace(',', '.').astype(float)

# Sortowanie wyników względem daty i czasu
data = data.sort_values(by=['Event Date', 'Time'])

# Inicjalizacja pustej listy do przechowywania najlepszych wyników
best_results = []

# Ustalenie daty początkowej
start_date = pd.to_datetime('1968-10-14', format='%Y-%m-%d')

# Przeszukiwanie posortowanych danych w celu znalezienia progresji rekordu świata
current_best_time = float('inf')
for _, row in data.iterrows():
    if row['Event Date'] >= start_date:
        if row['Time'] < current_best_time:
            best_results.append(row)
            current_best_time = row['Time']

# Konwersja listy wyników do DataFrame
best_results = pd.DataFrame(best_results)

# Tworzenie wykresu punktowego
fig = px.scatter(best_results, x='Event Date', y='Time', title='Progresja rekordu świata - 100m mężczyzn')
fig.update_traces(mode='markers+lines')  # Dodanie linii łączących punkty
#   height=700
fig.update_layout(
    xaxis=dict(title='Data biegu'),  # Zmiana etykiety osi x na "Czas"
    yaxis=dict(title='Czas'),  #
    height=600 )
# Zapisywanie wykresu do pliku HTML
fig.write_html("WR_progression.html")



In [13]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from PIL import Image

%matplotlib notebook

fig, ax = plt.subplots()
ax.set_xlim(0, 100)
ax.set_ylim(-1.5, 1.5)
red_point, = ax.plot([], [], 'ro', markersize=10)
blue_point, = ax.plot([], [], 'bo', markersize=10)

green_duration = 9.58  # Czas trwania dla zielonej kropki (Bolt) w sekundach
blue_duration = 14.55  # Czas trwania dla niebieskiej kropki (Average Person) w sekundach
fps = 30  # Liczba klatek na sekundę
green_frames = int(green_duration * fps)
blue_frames = int(blue_duration * fps)

# Inicjalizacja osi czasu
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, ha='left', va='top')

# Usunięcie siatki dla osi Y
ax.yaxis.set_visible(False)

# Podpisanie osi Y jako "Start"
ax.text(-10, 0, 'Start', ha='center', va='center', rotation='vertical')

# Dodanie kratki na końcu
ax.axvline(x=100, color='green', linestyle='--')

def update(frame):
    green_x = frame / green_frames * 100
    blue_x = frame / blue_frames * 100

    # Aktualizacja pozycji kropek
    red_point.set_data(green_x, -0.5)  # Zielona kropka dla Bolta
    blue_point.set_data(blue_x, 0.5)  # Niebieska kropka dla Average Person

    # Aktualizacja osi czasu
    elapsed_time = frame / fps
    time_text.set_text('Elapsed Time: {:.2f} s'.format(elapsed_time))

    # Zatrzymywanie zielonej kropki (Bolta) po 9.58 sekundach
    if elapsed_time >= green_duration:
        red_point.set_data([], [])  # Usunięcie zielonej kropki (Bolta)

    return red_point, blue_point, time_text

# Zmiana koloru linii dla Bolta na zielony
red_point.set_color('green')

# Dodanie legendy w prawym górnym rogu
ax.legend(['Usain Bolt', 'Average Person'], loc='upper right')

animation = FuncAnimation(fig, update, frames=blue_frames, interval=1000/fps, blit=True)

# Zapisywanie animacji jako pliku GIF
animation.save('animation.gif', writer='pillow')

plt.show()

<IPython.core.display.Javascript object>

/tmp/ipykernel_8102/635650999.py:36: MatplotlibDeprecationWarning:

Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later

/tmp/ipykernel_8102/635650999.py:37: MatplotlibDeprecationWarning:

Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later



##### DONE PRZEROBIONE Z MATPLOTLIB NA PPLOTLY - KUBA ######

In [14]:
import csv
from datetime import datetime
import re
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def calculate_age(birth_date, event_date):
    birth_date = datetime.strptime(birth_date, "%d.%m.%y")
    event_date = datetime.strptime(event_date, "%d.%m.%Y")
    
    birth_year = birth_date.year
    event_year = event_date.year
    
    if birth_year > 2010:
        birth_year = birth_year - 100
    
    age = event_year - birth_year
    if event_date.month < birth_date.month or (event_date.month == birth_date.month and event_date.day < birth_date.day):
        age -= 1
    return age


def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)


# Wczytanie danych z pliku CSV
data = pd.read_csv('men_100m.csv')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.replace(',', '.').astype(float)

# Znalezienie najlepszego czasu dla każdego zawodnika
best_times = data.groupby('Athlete')['Time'].min()

all_ages = []
# Iteracja po zawodnikach i ich najlepszych czasach
for athlete, best_time in best_times.items():
    athlete_data = data[(data['Athlete'] == athlete) & (data['Time'] == best_time)]
    
    birth_date = athlete_data['Date of Birth'].values[0]
    event_date = athlete_data['Event Date'].values[0]

    # Sprawdzenie, czy birth_date lub event_date jest puste lub nie jest prawidłową datą
    if pd.isnull(birth_date) or pd.isnull(event_date) or not isinstance(birth_date, str) or not isinstance(event_date, str):
        continue
    
    try:
        age = calculate_age(birth_date, event_date)
        all_ages.append(age)
    except ValueError:
        continue

unique_values, frequencies = np.unique(all_ages, return_counts=True)
value_to_delete = 83

# Znalezienie indeksu wartości do usunięcia
index_to_delete = np.where(unique_values == value_to_delete)[0]
unique_values = np.delete(unique_values, index_to_delete)
frequencies = np.delete(frequencies, index_to_delete)

# Tworzenie wykresu w Plotly
fig = go.Figure()

# Dodawanie słupków na wykres
fig.add_trace(go.Bar(
    x=unique_values,
    y=frequencies,
    marker_color='#4287f5',  # Kolor słupków
    text=frequencies,
    textposition='outside',  # Wyświetlanie tekstu nad słupkiem
    textfont=dict(size=12, color='black')  # Konfiguracja czcionki tekstu
))

# Konfiguracja osi i tytułu wykresu
fig.update_layout(
    xaxis_title='Wiek biegacza',
    yaxis_title='Liczba biegaczy, którzy ustawili rekord osobisty',
    title='Wiek vs Personal Best',
    xaxis=dict(type='linear'),  # Wymuszenie liniowego układu osi X
    yaxis=dict(showgrid=True, gridcolor='lightgray'),  # Wyświetlanie siatki na osi Y
    height=700
)

# Zapisywanie wykresu w pliku HTML
fig.write_html('personal_best_graph.html')



In [19]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Wczytanie danych z pliku CSV
df = pd.read_csv('heatmap_runners.csv')

# Tworzenie macierzy danych
heatmap_data = []
athletes = df['Athlete'].unique()
ages = range(18, 41)  # Zakres wieku od 18 do 40
for athlete in athletes:
    row = []
    for age in ages:
        birth_year = df.loc[df['Athlete'] == athlete, 'Date of Birth'].values[0]
        time = df.loc[np.logical_and(df['Date of Birth'] + age == df['Run Date'], df['Athlete'] == athlete), 'Time'].values
        row.append(time[0] if len(time) > 0 else None)
    heatmap_data.append(row)

# Tworzenie heatmapy w Plotly z dostosowaną skalą kolorów
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data,
    x=list(ages),
    y=athletes,
    colorscale='Reds',  # Zmiana skali kolorów na 'Reds'
    reversescale=True,  # Odwrócenie skali kolorów
    xgap=0.1,  # Odstęp między prostokątami na osi X
))

# Konwertowanie wartości na osi Y na typ numeryczny
numeric_athletes = [i for i in range(len(athletes))]

# Konfiguracja osi i tytułu wykresu
fig.update_layout(
    xaxis_title='Wiek',
    yaxis_title='Zawodnik',
    title='Najlepsze wyniki sprinterów w zależności od wieku',
    yaxis=dict(
        tickmode='array',
        tickvals=numeric_athletes,
        ticktext=athletes,
        range=[numeric_athletes[0]-0.5, numeric_athletes[-1]+0.5],  # Ograniczenie zakresu osi Y
    ),
    height=500
)




fig.write_html('heatmap.html')


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Wczytywanie danych z pliku CSV
df = pd.read_csv('men_100m.csv')

# Konwersja kolumny 'Wind' na typ float
df['Wind'] = pd.to_numeric(df['Wind'], errors='coerce')

# Usunięcie literek "A" na końcu niektórych wartości czasu
df['Time'] = df['Time'].str.rstrip('A')

# Usunięcie duplikatów
df = df.drop_duplicates(subset=['Time', 'Wind'])

# Podział danych na dodatni i ujemny wiatr
positive_wind = df[df['Wind'] >= 0]
negative_wind = df[df['Wind'] < 0]

# Tworzenie wykresu
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=positive_wind['Time'],
    y=positive_wind['Wind'],
    mode='markers',
    marker=dict(
        size=8,
        color='red'  # Zmiana koloru na czerwony
    ),
    name='Wiatr pozytywny'
))

fig.add_trace(go.Scatter(
    x=negative_wind['Time'],
    y=negative_wind['Wind'],
    mode='markers',
    marker=dict(
        size=8,
        color='blue'  # Zmiana koloru na niebieski
    ),
    name='Wiatr negatywny'
))

# Ustawienia osi
fig.update_layout(
    title='Zależność między wiatrem a czasem',
    xaxis=dict(title='Czas'),
    yaxis=dict(title='Wiatr', range=[-2.5, 2.5]),
    height=700
)


# Wyświetlanie wykresu
fig.write_html('wind.html')


#### DEVELOPMENT MODE - MARIUSZ ###

In [ ]:
%pip install pycountry

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv
import pycountry
import pandas as pd
# Create a set to store unique athletes and their countries
import plotly.express as px
athlete_country_set = set()

# Read the CSV file
with open('men_100m.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row

    # Iterate through each row in the CSV
    for row in csv_reader:
        athlete = row[3]
        country_code = row[4]
        if country_code == 'GDR':
            country_code = 'DEU'
        if country_code == 'GER':
           country_code = 'DEU'  
        if country_code == 'RSA':
           country_code =  'ZAF'   
        if country_code == 'NGR':
           country_code =  'NGA'
        if country_code == 'POR':
           country_code =  'PRT'     
        if country_code == 'BAR':
           country_code =  'BRB' 
        if country_code == 'ZIM':
           country_code =  'ZWE'         
        if country_code == 'BAH':
           country_code =  'BHS'   
        if country_code == 'ANT':
           country_code =  'ATG'   
        if country_code == 'NED':
           country_code =  'NLD'   
        if country_code == 'BOT':
           country_code =  'BWA'   
        if country_code == 'CAY':
           country_code =  'CYM'   
        if country_code == 'SKN':
           country_code =  'KNA'   
        if country_code == 'AHO':
           country_code =  'NLD'   
        if country_code == 'SRI':
           country_code =  'LKA'   
        if country_code == 'OMA':
           country_code =  'OMN'   
        if country_code == 'SUI':
           country_code =  'CHE'   
        if country_code == 'MAS':
           country_code =  'MYS'   
        if country_code == 'KSA':
           country_code =  'SAU'       
        if country_code == 'IRI':
           country_code =  'IRN'  
        if country_code == 'ZAM':
           country_code =  'ZMB'  


        athlete_country_set.add((athlete, country_code))

# Create a dictionary to map country codes to country names
#print(athlete_country_set)
country_names = {}

# Replace country codes with country names
for athlete, country_code in athlete_country_set:
    try:
        country = pycountry.countries.get(alpha_3=country_code)
        if country:
            country_names[country_code] = country.name
    except LookupError:
        country_names[country_code] = 'Unknown'

# Print each unique athlete with their full country name

country_counts = {}
for _, country_code in athlete_country_set:
    if country_code in country_counts:
        country_counts[country_code] += 1
    else:
        country_counts[country_code] = 1

# Create a DataFrame from the country counts
df = pd.DataFrame.from_dict(country_counts, orient='index', columns=['Liczba Biegaczy'])
df['Country'] = df.index

# Create the choropleth map
fig = px.choropleth(df, locations='Country', locationmode='ISO-3', color='Liczba Biegaczy',
                    title='Liczba biegaczy według kraju', color_continuous_scale='YlOrRd', )
fig.update_layout(
    title_font_size=20,  # Increase the title font size
    font_size=12,  # Set the overall font size
    geo=dict(
        showframe=False,  # Hide the map frame
        showcoastlines=False,  # Hide the coastlines
        projection_type='natural earth'  # Use the natural earth projection
    ),
    height=700,  # Set the height of the graph (in pixels)
    width=1220  # Set the width of the graph (in pixels)
)
# Show the map
fig.show()

fig.write_html("Runners_per_country.html")
